In [1]:
!apt-get install swig
!pip install box2d box2d-py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 21 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 2s (495 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubunt

In [2]:
pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.5 MB/s eta 0:00:00
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp311-cp311-linux_x86_64.whl size=2351224 sha256=5ba771b790dd6bff517706571c2891d77975b07ecea079145a5a0adda014e019
  Stored in directory: /root/.cache/pip/wheels/ab/f1/0c/d56f4a2bdd12bae0a0693ec33f2f0daadb5eb9753c78fa5308
Successfully built box2d-py
  Attempting uninstall: box2d-py
    Found existing installation: box2d-py 2.3.8
    Uninstalling box2d-py-2.3.8:
      Successfully uninstalled box2d-py-2.3.8


In [ ]:
pip install wandb

In [1]:
#importing dependencies
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

# this class define the storage buffer of the environment

class StorageBuffer:
    def __init__(self):
        self.reset()
        self.episode_rewards = []  # Store rewards for each episode
        self.episode_lengths = []  # Store lengths for each episode

    def reset(self):
        # Current episode storage
        self.states = []
        self.actions = []
        self.rewards = []
        self.log_probs = []
        self.dones = []
        self.next_state = None
        self.current_reward = 0

    def add_step(self, state, action, reward, log_prob, done, next_state=None):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.log_probs.append(log_prob)
        self.dones.append(done)
        self.current_reward += reward

        if done:
            self.next_state = next_state
            self.episode_rewards.append(self.current_reward)
            self.episode_lengths.append(len(self.rewards))

    def get_episode_data(self):
        return {
            'states': torch.FloatTensor(np.array(self.states)),
            'actions': torch.tensor(self.actions),
            'rewards': torch.tensor(self.rewards),
            'log_probs': torch.tensor(self.log_probs),
            'dones': torch.tensor(self.dones),
            'next_state': torch.FloatTensor(self.next_state).unsqueeze(0) if self.next_state is not None else None
        }

    def get_statistics(self):

        if not self.episode_rewards:
            return {"mean_reward": 0, "max_reward": 0, "min_reward": 0, "mean_length": 0}

        return {
            "mean_reward": np.mean(self.episode_rewards),
            "max_reward": np.max(self.episode_rewards),
            "min_reward": np.min(self.episode_rewards),
            "mean_length": np.mean(self.episode_lengths),
            "current_reward": self.current_reward,
            "current_length": len(self.rewards)
        }

#the policy and value networks
# i used small model since it simple game but you can make it bigger if you want

class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(PolicyNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, action_dim),
        )
        self.log_std = nn.Parameter(torch.zeros(action_dim))

    def forward(self, x):
        mean =   self.fc(x)
        std = self.log_std.exp()
        return mean,std

class ValueNetwork(nn.Module):
    def __init__(self, state_dim):
        super(ValueNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.fc(x)


In [2]:
from torch.distributions import Normal
class PPOAgent:
    def __init__(self, state_dim, action_dim, lr=3e-4, gamma=0.99, eps_clip=0.2, k_epochs=10):

        self.policy = PolicyNetwork(state_dim, action_dim)
        self.old_policy = PolicyNetwork(state_dim, action_dim)
        self.old_policy.load_state_dict(self.policy.state_dict())
        self.value_net = ValueNetwork(state_dim)

        self.policy_optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.value_optimizer = optim.Adam(self.value_net.parameters(), lr=lr)

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.k_epochs = k_epochs
        self.mse_loss = nn.MSELoss()

    def select_action(self, state):
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            mean, std = self.old_policy(state)

        dist = Normal(mean, std)
        action = dist.sample()
        action_log_prob = dist.log_prob(action).sum(dim=-1)  # Sum for multi-dimensional actions
        return action.numpy()[0], action_log_prob.numpy()[0]

    def compute_advantages(self, rewards, values, next_value, dones, lambda_gae=0.95):
        advantages = []
        gae = 0
        for t in reversed(range(len(rewards))):
            delta = rewards[t] + self.gamma * (1 - float(dones[t])) * next_value - values[t]
            gae = delta + self.gamma * lambda_gae * (1 - float(dones[t])) * gae
            advantages.insert(0, gae)
            next_value = values[t]
        advantages = torch.tensor(advantages, dtype=torch.float32)
        return (advantages - advantages.mean()) / (advantages.std() + 1e-8)


    def save_model(self, filepath):
        torch.save({
            'policy_state_dict': self.policy.state_dict(),
            'old_policy_state_dict': self.old_policy.state_dict(),
            'value_state_dict': self.value_net.state_dict(),
            'policy_optimizer_state_dict': self.policy_optimizer.state_dict(),
            'value_optimizer_state_dict': self.value_optimizer.state_dict()
        }, filepath)

    def load_model(self, filepath):
        checkpoint = torch.load(filepath)
        self.policy.load_state_dict(checkpoint['policy_state_dict'])
        self.old_policy.load_state_dict(checkpoint['old_policy_state_dict'])
        self.value_net.load_state_dict(checkpoint['value_state_dict'])
        self.policy_optimizer.load_state_dict(checkpoint['policy_optimizer_state_dict'])
        self.value_optimizer.load_state_dict(checkpoint['value_optimizer_state_dict'])

        # Ensure all models are in eval mode for testing
        self.policy.eval()
        self.old_policy.eval()
        self.value_net.eval()

    def update(self, buffer, batch_size=64):
        data = buffer.get_episode_data()
        states = torch.tensor(data['states'], dtype=torch.float32)
        actions = torch.tensor(data['actions'], dtype=torch.float32)
        log_probs_old = torch.tensor(data['log_probs'], dtype=torch.float32)
        rewards = data['rewards']
        dones = data['dones']

        values = self.value_net(states).squeeze().detach()
        next_value = self.value_net(torch.tensor(data['next_state'], dtype=torch.float32)).item() if data['next_state'] else 0.0
        advantages = self.compute_advantages(rewards, values, next_value, dones)
        targets = advantages + values

        dataset = torch.utils.data.TensorDataset(states, actions, log_probs_old, advantages, targets)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

        for _ in range(self.k_epochs):
            for batch_states, batch_actions, batch_log_probs_old, batch_advantages, batch_targets in dataloader:

                mean, std = self.policy(batch_states)
                values_pred  =  self.value_net(batch_states)
                dist = Normal(mean, std)
                log_probs = dist.log_prob(batch_actions).sum(dim=-1)

                ratios = torch.exp(log_probs - batch_log_probs_old)
                surr1 = ratios * batch_advantages
                surr2 = torch.clamp(ratios, 1 - self.eps_clip, 1 + self.eps_clip) * batch_advantages
                loss_actor = -torch.min(surr1, surr2).mean()

                loss_critic = self.mse_loss(values_pred.squeeze(), batch_targets)

                self.policy_optimizer.zero_grad()
                loss_actor.backward(retain_graph=True)
                self.policy_optimizer.step()

                self.value_optimizer.zero_grad()
                loss_critic.backward()
                self.value_optimizer.step()

        self.old_policy.load_state_dict(self.policy.state_dict())


In [3]:
import statistics
def train_ppo(env_name="BipedalWalker-v3", num_episodes=2000, max_timesteps=2000):
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    agent = PPOAgent(state_dim, action_dim)
    buffer = StorageBuffer()
    best_reward = -float('inf')

    mean_raward = []
    for episode in range(num_episodes):
        state, _ = env.reset()
        buffer.reset()
        episode_reward = 0

        for t in range(max_timesteps):
            action, log_prob = agent.select_action(state)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            episode_reward += reward

            buffer.add_step(state, action, reward, log_prob, done, next_state if not done else None)
            state = next_state
            if done:
                break
        mean_raward.append(episode_reward)
        # wandb.log({"episode_reward":episode_reward})

        agent.update(buffer)
        if episode_reward > best_reward:
            best_reward = episode_reward
            agent.save_model("bipedalwalker.pth")

        if (episode + 1) % 10 == 0:
            print(f"Episode {episode + 1}, mean raward: {statistics.mean(mean_raward)}, Best: {best_reward:.2f}")
            mean_raward = []

    env.close()
    return agent


def test_model(model_path, env_name="BipedalWalker-v3", num_episodes=10):
    env = gym.make(env_name, render_mode="human")
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    agent = PPOAgent(state_dim, action_dim)
    agent.load_model(model_path)

    for episode in range(num_episodes):
        state, _ = env.reset()
        done = False
        total_reward = 0

        while not done:
            action, _ = agent.select_action(state)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += reward
            state = next_state

        print(f"Test Episode {episode + 1}, Total Reward: {total_reward:.2f}")

    env.close()


In [6]:
train_ppo()

<ipython-input-3-46d81eb0ba57>:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  'actions': torch.tensor(self.actions),
<ipython-input-4-117b7c798710>:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  states = torch.tensor(data['states'], dtype=torch.float32)
<ipython-input-4-117b7c798710>:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  actions = torch.tensor(data['actions'], dtype=torch.float32)
<ipython-input-4-117b7c798710>:66: UserWarning: To copy construct from a tensor,

Episode 10, mean raward: -99.93580009136424, Best: -72.34
Episode 20, mean raward: -112.6236916339224, Best: -72.34
Episode 30, mean raward: -116.88318935659956, Best: -70.27


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Episode 40, mean raward: -115.28107283723963, Best: -70.27
Episode 50, mean raward: -116.9066574259234, Best: -70.27
Episode 60, mean raward: -113.79672936876932, Best: -70.27
Episode 70, mean raward: -116.17572300497957, Best: -70.27
Episode 80, mean raward: -110.02570149639239, Best: -70.27
Episode 90, mean raward: -91.84736049857655, Best: -68.72
Episode 100, mean raward: -116.15181642928626, Best: -68.72
Episode 110, mean raward: -100.82962420187887, Best: -68.72
Episode 120, mean raward: -101.63209073847503, Best: -68.72
Episode 130, mean raward: -106.48455070313408, Best: -68.72
Episode 140, mean raward: -108.48783668053439, Best: -68.72
Episode 150, mean raward: -105.99701019759694, Best: -68.72
Episode 160, mean raward: -104.65312283607486, Best: -68.72
Episode 170, mean raward: -104.33744079761257, Best: -68.72
Episode 180, mean raward: -105.23578079898947, Best: -68.72
Episode 190, mean raward: -112.47844194968144, Best: -68.72
Episode 200, mean raward: -114.34771116834094, B

In [4]:
model_path  = "bipedalwalker.pth"
test_model(model_path)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9224\4034181294.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filepath)


Test Episode 1, Total Reward: 110.19
Test Episode 2, Total Reward: 116.27
Test Episode 3, Total Reward: 110.63
Test Episode 4, Total Reward: 114.90
Test Episode 5, Total Reward: -17.01
Test Episode 6, Total Reward: 109.05
Test Episode 7, Total Reward: 83.73
Test Episode 8, Total Reward: 106.99
Test Episode 9, Total Reward: 96.32
Test Episode 10, Total Reward: 103.72


False
False
